## Importing libraries

In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing dataset

In [3]:
root_dir = "/content/drive/MyDrive/Datasets/Recommender_Systems/Movies_Dataset"

movies = pd.read_csv(root_dir+'/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
print(movies.head())  # Movie_id, Movie_name, Movie_genre

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy


In [4]:
users = pd.read_csv(root_dir+'/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
print(users.head())     # User_id, Gender, Age, _, zip_code

   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7  02460
4  5  M  25  20  55455


In [6]:
ratings = pd.read_csv(root_dir+'/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
print(ratings.head())   # User_id, movie_id, rating(1 to 5), timestamp

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291


## Preparing the training set and the test set

In [12]:
training_set = pd.read_csv(root_dir+'/ml-100k/u1.base', delimiter="\t")
print(training_set.head()) # User_id, Movie_id, rating, timestamp

# convert dataframe to numpy array
training_set = np.array(training_set, dtype='int')
print(f'Shape of training_set: {training_set.shape}')

   1  1.1  5  874965758
0  1    2  3  876893171
1  1    3  4  878542960
2  1    4  3  876893119
3  1    5  3  889751712
4  1    7  4  875071561
Shape of training_set: (79999, 4)


In [20]:
test_set = pd.read_csv(root_dir+'/ml-100k/u1.test', delimiter="\t")
print(test_set.head()) # User_id, Movie_id, rating, timestamp

# convert dataframe to numpy array
test_set = np.array(test_set, dtype='int')
print(f'Shape of testing_set: {test_set.shape}')

   1   6  5  887431973
0  1  10  3  875693118
1  1  12  5  878542960
2  1  14  5  874965706
3  1  17  3  875073198
4  1  20  4  887431883
Shape of testing_set: (19999, 4)


## Getting the number of users and movies

In [16]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
print(f'Maximum number of users: {nb_users}')
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(f'Maximum number of movies: {nb_movies}')

Maximum number of users: 943
Maximum number of movies: 1682


## Converting the data into an array with users in lines and movies in columns

In [27]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users+1):
    id_movies = data[:, 1][data[:,0] == id_users]
    id_ratings = data[:, 2][data[:,0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [34]:
training_set = convert(training_set)
test_set = convert(test_set)

In [38]:
print(training_set[:10])
print(test_set[:10])

[[0.0, 3.0, 4.0, 3.0, 3.0, 0.0, 4.0, 1.0, 5.0, 0.0, 2.0, 0.0, 5.0, 0.0, 5.0, 5.0, 0.0, 4.0, 5.0, 0.0, 1.0, 4.0, 0.0, 0.0, 4.0, 3.0, 0.0, 4.0, 1.0, 3.0, 0.0, 5.0, 0.0, 2.0, 1.0, 0.0, 2.0, 3.0, 0.0, 3.0, 2.0, 5.0, 4.0, 0.0, 5.0, 4.0, 0.0, 5.0, 0.0, 5.0, 0.0, 4.0, 0.0, 0.0, 5.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 5.0, 4.0, 5.0, 0.0, 0.0, 0.0, 5.0, 2.0, 4.0, 0.0, 0.0, 0.0, 3.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 4.0, 0.0, 0.0, 5.0, 1.0, 5.0, 0.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 5.0, 0.0, 0.0, 3.0, 4.0, 5.0, 0.0, 2.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, 4.0, 0.0, 4.0, 3.0, 5.0, 1.0, 3.0, 0.0, 3.0, 2.0, 0.0, 4.0, 0.0, 4.0, 3.0, 0.0, 2.0, 0.0, 0.0, 5.0, 3.0, 0.0, 0.0, 4.0, 0.0, 3.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 5.0, 5.0, 2.0, 5.0, 5.0, 0.0, 0.0, 5.0, 5.0, 0.0, 0.0, 5.0, 0.0, 5.0, 3.0, 0.0, 5.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 5.0, 4.0, 0.0, 4.0, 5.0, 0.0, 5.0, 5.0, 4.0, 0.0

In [39]:
print(len(training_set))

943


## Converting the data into torch tensors

In [40]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [49]:
# Hyperparameters
lr = 0.01
epochs = 200

## Creating the architecture of our Stacked AutoEncoders

In [41]:
class SAE(nn.Module):
  def __init__(self):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x = self.sigmoid(self.fc1(x))
    x = self.sigmoid(self.fc2(x))
    x = self.sigmoid(self.fc3(x))
    x = self.fc4(x)
    return x

In [42]:
model = SAE()

In [48]:
# Creating loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=0.5) 
# weight_decay is used to reduce the lr after every few epochs in order to regulate the convergence.

## Training our model

In [53]:
for epoch in range(1, epochs+1):
  train_loss = 0.0
  s = 0.0
  for id_users in range(nb_users):
    # batch of single input vector
    # online learning: update the weights after each observation going through the network 
    input = Variable(training_set[id_users]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = model(input)
      target.requires_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
      optimizer.step()
  print(f'Epoch: {epoch}/{epochs}\tLoss: {train_loss/s}')

Epoch: 1/200	Loss: 1.7666395940458108
Epoch: 2/200	Loss: 1.09648219072975
Epoch: 3/200	Loss: 1.053370260510873
Epoch: 4/200	Loss: 1.0384998490007624
Epoch: 5/200	Loss: 1.030879691406409
Epoch: 6/200	Loss: 1.0265496544594603
Epoch: 7/200	Loss: 1.0237601500288143
Epoch: 8/200	Loss: 1.022026077530451
Epoch: 9/200	Loss: 1.0206960270817578
Epoch: 10/200	Loss: 1.019673500398135
Epoch: 11/200	Loss: 1.0188573404948476
Epoch: 12/200	Loss: 1.0184826747616929
Epoch: 13/200	Loss: 1.0177940103492924
Epoch: 14/200	Loss: 1.0176849122586729
Epoch: 15/200	Loss: 1.0171108811725862
Epoch: 16/200	Loss: 1.0169381738536998
Epoch: 17/200	Loss: 1.0168200632069029
Epoch: 18/200	Loss: 1.0165293730720075
Epoch: 19/200	Loss: 1.0164695691712993
Epoch: 20/200	Loss: 1.0161735691563558
Epoch: 21/200	Loss: 1.0162806603309327
Epoch: 22/200	Loss: 1.0159780272664247
Epoch: 23/200	Loss: 1.0158228417191466
Epoch: 24/200	Loss: 1.0157306442078464
Epoch: 25/200	Loss: 1.0157860903571527
Epoch: 26/200	Loss: 1.0156545999559885
E

## Testing our model

In [54]:
test_loss = 0.0
s = 0.0
for id_users in range(nb_users):
  # batch of single input vector
  # online learning: update the weights after each observation going through the network 
  input = Variable(training_set[id_users]).unsqueeze(0)
  target = Variable(test_set[id_users]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = model(input)
    target.requires_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item()*mean_corrector)
    s += 1.
print(f'Test Loss: {test_loss/s}')

Test Loss: 0.9609286811309583
